In [ ]:
# default_exp cli

In [2]:
# export
from fastcore.all import *
from bs4 import BeautifulSoup
from pathlib import Path
from functools import partial
from shutil import copyfile
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.remote.webdriver import WebDriver as RemoteWebDriver
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from cryptography.fernet import Fernet
from selenium.webdriver.common.keys import Keys
from itertools import count
from time import sleep
import http
import webbrowser
import shutil
import flask
import os
import click
import pyperclip
import colorama
import pickle
import json

from play_it_later.encryption import *
from play_it_later.talker import *

In [3]:
# export
@call_parse
def encrypt_game(p:Param('The path to the game save.',Path),
                 year:Param('year to decrypt',int),month:Param('month to decrypt',int),
                 day:Param('day to decrypt',int),hour:Param('hour to decrypt',int),minute:Param('minute to decrypt',int),
                 block_sz:Param('The block size to encrypt the file. Should be multiples of 8',int)=65536,
                 verbose:Param('print debug print statements. Will show the original key.',bool_arg)=False):
    key=Fernet.generate_key()
    if verbose:print(key)
    if str(p)[0]!='/': Path(os.getcwd())/p
    process_dirs(p,partial(encrypt_file,key=key,block_sz=block_sz,verbose=verbose))
    key=send_encryption_key(key,year,month,day,hour,minute)
    txt2encrypted_file(p.parent/'encrypted'/p.parts[-1],key)

In [4]:
!encrypt_game test_save 2020 12 11 18 15

 
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/play_it_later/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache
Please pick from these days in month 12: (12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31)


In [5]:
# export
@call_parse
def decrypt_game(p:Param('The path to the game save.',Path),
                 prefix:Param('Where to put the decrypted file.',str),
                 verbose:Param('print debug print statements. Will show the original key.',bool_arg)=False,
                 mv_original_dir:Param('Move to the original directory.',bool_arg)=False):
    if str(p)[0]!='/': Path(os.getcwd())/p
    en_p=p if 'encrypted' in p.parts else p.parent/'encrypted'/p.parts[-1]
    key=encrypted_file2txt(en_p)
    key,result_description=request_decryption_key(key)
    if verbose:print(key)
    print(result_description)
    if key=='': 
        print('key is empty. This is likely due to it being locked.')
        return
    process_dirs(en_p,partial(decrypt_file,key=key,verbose=verbose,prefix=prefix))
    
    if mv_original_dir:
        shutil.move(str(en_p.parents[2]/prefix/en_p if len(en_p.parts)>2 else prefix/en_p),p)

In [6]:
!decrypt_game test_save decrypted --verbose y

 
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/play_it_later/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache
The message was encrypted to be inaccessible until 12/12/2020, 6:15:00 PM local time or Sat, 12 Dec 2020 18:15:00 GMT UTC
Traceback (most recent call last):
  File "/opt/conda/envs/play_it_later/bin/decrypt_game", line 33, in <module>
    sys.exit(load_entry_point('play-it-later', 'console_scripts', 'decrypt_game')())
  File "/opt/conda/envs/play_it_later/lib/python3.7/site-packages/fastcore/script.py", line 103, in _f
    tfunc(**merge(args, args_from_prog(func, xtra)))
  File "/opt/project/play_it_later/play_it_later/cli.py", line 61, in decrypt_game
    key,result_description=request_decryption_key(key)
  File "/opt/project/play_it_later/play_it_later/talker.py", line 133, in request_decryption_key
    if i==max_tries: raise HttpConnectionFailure('Failed to get decrypted_ms

In [7]:
# hide
from nbdev.export import *
from nbdev.export2html import *
from nbdev.cli import make_readme
make_readme()
notebook2script()
notebook2html()

converting /opt/project/play_it_later/nbs/index.ipynb to README.md
Converted 00_encryption.ipynb.
Converted 01_talker.ipynb.
Converted 02_cli.ipynb.
Converted index.ipynb.
No notebooks were modified
